# Assignment 3 Question 1

### CO 487/687 Applied Cryptography Fall 2024

This Jupyter notebook contains Python 3 code for Assignment 3 Question 1 on "Symemtric Encryption in Python".

### Documentation

- [Python cryptography library](https://cryptography.io/en/latest/)

The following code imports all the required functions for the assignment.

In [10]:
import base64
import getpass
import json
import os
import sys
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes    
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives import constant_time
from timeit import default_timer as timer
from cryptography.exceptions import InvalidSignature

Sometimes you need to convert between printable strings and byte arrays, because cryptographic routines often work with byte arrays. 

If you need to take a user-supplied string (like a message or a password) and convert it into a byte array, you can do so like this:

In [11]:
message = "Hello, world!"
message_as_byte_array = message.encode('utf-8')
message_back_to_string = message_as_byte_array.decode('utf-8')
print(message_back_to_string)

Hello, world!


If you need to take byte array (such as a cryptographic key or hash output or ciphertext) and convert it to a string for printing, and then convert that string back to a byte array, you can use the following functions:

In [12]:
def bytes2string(b):
    return base64.urlsafe_b64encode(b).decode('utf-8')

def string2bytes(s):
    return base64.urlsafe_b64decode(s.encode('utf-8'))

sample_byte_array = b'\x01\x23\x45\x67\x89\xAB\xCD\xEF'
sample_encoded_as_string = bytes2string(sample_byte_array)
print(sample_encoded_as_string)
sample_back_to_byte_array = string2bytes(sample_encoded_as_string)
assert sample_back_to_byte_array == sample_byte_array

ASNFZ4mrze8=


Implement the main encryption function below. Your function will take as input a string, and will output a string or dictionary containing all the values needed to decrypt (other than the password, of course). The code below will prompt the user to enter their password during encryption.

In [13]:
def encrypt(message):
    
    # encode the string as a byte string, since cryptographic functions usually work on bytes
    plaintext = message.encode('utf-8')

    # Use getpass to prompt the user for a password
    password = getpass.getpass("Enter password:")
    password2 = getpass.getpass("Enter password again:")

    # Do a quick check to make sure that the password is the same!
    if password != password2:
        sys.stderr.write("Passwords did not match")
        sys.exit()

    ### START: This is what you have to change
    
    # Derive key using Scrypt
    salt = os.urandom(16)
    kdf = Scrypt(
        salt=salt,
        length=32,
        n=2**14,
        r=8,
        p=1
    )
    key = kdf.derive(password.encode('utf-8'))

    # Initialize AES in cipher feedback mode
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv))
    encryptor = cipher.encryptor()

    # Encrypt the plaintext
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()

    # Create HMAC for integrity
    h = hmac.HMAC(key, hashes.SHA3_512())
    h.update(ciphertext)
    hmac_tag = h.finalize()

    # Return all components as a JSON string
    return json.dumps({
        'salt': bytes2string(salt),
        'iv': bytes2string(iv),
        'ciphertext': bytes2string(ciphertext),
        'hmac': bytes2string(hmac_tag)
    })

    ### END: This is what you have to change

Now we call the `encrypt` function with a message, and print out the ciphertext it generates.

In [14]:
mymessage = "Hello, world!"
ciphertext = encrypt(mymessage)
print(ciphertext)

{"salt": "HUJ0WD8shToXNz02mj72wA==", "iv": "ZFDCZsjYO7VaCZVzLiEyEg==", "ciphertext": "plkk4hbf7yROw78HSA==", "hmac": "yfkAkz56andQYc3L3DCZJIeBGIALKRXu4rC913zvh3AYPs96RjLLbSQX-VNrBhiPmkazD2TwiYlpW0nvusR2-w=="}


Implement the main decryption function below.  Your function will take as input the string or dictionary output by `encrypt`, prompt the user to enter the password, and then do all the relevant cryptographic operations.

In [15]:
def decrypt(ciphertext):
    
    # prompt the user for the password
    password = getpass.getpass("Enter the password:")

    ### START: This is what you have to change

     # Parse the JSON string to extract the components
    data = json.loads(ciphertext)
    salt = string2bytes(data['salt'])
    iv = string2bytes(data['iv'])
    encrypted_text = string2bytes(data['ciphertext'])
    hmac_tag = string2bytes(data['hmac'])

    # Derive key using Scrypt
    kdf = Scrypt(
        salt=salt,
        length=32,
        n=2**14,
        r=8,
        p=1
    )
    key = kdf.derive(password.encode('utf-8'))

    # Verify HMAC for integrity
    h = hmac.HMAC(key, hashes.SHA3_512())
    h.update(encrypted_text)
    try:
        h.verify(hmac_tag)
    except InvalidSignature:
        sys.stderr.write("Invalid password or tampered data\n")
        sys.exit()

    # Decrypt using AES in CFB mode
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv))
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(encrypted_text) + decryptor.finalize()

    # Return the decoded plaintext
    return plaintext.decode('utf-8')

Now let's try decrypting the ciphertext you encrypted above by entering the same password as you used for encryption.

In [16]:
mymessagedecrypted = decrypt(ciphertext)
print(mymessagedecrypted)
assert mymessagedecrypted == mymessage

Hello, world!


Try again but this time see what happens if you use a different password to decrypt. Your function should fail.

In [17]:
mymessagedecrypted = decrypt(ciphertext)
print(mymessagedecrypted)
assert mymessagedecrypted == mymessage

Hello, world!


If you would like to measure the runtime of a particular operation, you can use the following snippit of code:

In [18]:
start = timer()
YOUR_OPERATION_HERE()
end = timer()
print(f"runtime: {end-start} seconds")

NameError: name 'YOUR_OPERATION_HERE' is not defined